### Importação de configurações e funções

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *

apply_storage_config(spark)

In [0]:
from pyspark.sql.functions import (
    col,
    when,
    lit,
    trim,
    upper,
    lower,
    coalesce,
    concat,
    substring,
    length,
    cast,
    round,
    sum as spark_sum,
    count,
    avg,
    max as spark_max,
    min as spark_min,
    countDistinct,
    current_timestamp,
    year,
    month,
    datediff,
    to_date,
    date_format,
    rank,
    dense_rank,
    lag,
    lead,
    sha2,
    concat_ws,
    ceil,
    row_number
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType
)

from pyspark.sql import Window

### Definição dos paths

In [0]:
path_storage_silver = f"{silver_path}"
path_storage_gold = f"{gold_path}"


### Criação da tabela de controle de transformações Gold

In [0]:
def criar_tabela_controle_gold():
    """
    Cria tabela para rastrear transformações realizadas na Gold
    """
    spark.sql("""
        CREATE TABLE IF NOT EXISTS gold.transformacoes_processadas_cnpj (
            tabela_gold STRING,
            data_ultima_transformacao TIMESTAMP,
            total_registros BIGINT,
            data_processamento TIMESTAMP
        )
        USING DELTA
    """)
    print("Tabela de controle de transformações Gold CNPJ criada!")

criar_tabela_controle_gold()

### Funções auxiliares para transformações

In [0]:
def obter_data_ultima_transformacao_gold(tabela_gold):
    """
    Obtém a data da última transformação realizada para uma tabela na Gold
    """
    try:
        resultado = spark.sql(f"""
            SELECT MAX(data_ultima_transformacao) as ultima_data
            FROM gold.transformacoes_processadas_cnpj
            WHERE tabela_gold = '{tabela_gold}'
        """).collect()
        
        if resultado and resultado[0].ultima_data:
            return resultado[0].ultima_data
        else:
            return None
    except:
        return None

In [0]:
def registrar_transformacao_gold(tabela_gold, total_registros):
    """
    Registra transformação realizada na Gold
    """
    spark.sql(f"""
        INSERT INTO gold.transformacoes_processadas_cnpj
        VALUES (
            '{tabela_gold}',
            current_timestamp(),
            {total_registros},
            current_timestamp()
        )
    """)

In [0]:
%sql
select * from gold.transformacoes_processadas_cnpj

In [0]:
dbutils.fs.ls(f"{path_storage_silver}/cnpj/Estabelecimentos_tratado/")

### Dimensão TEMPO_ABERTURA

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO TEMPO_ABERTURA")
print("=" * 60)

# Ler dados de estabelecimentos da Silver
df_estabelecimentos = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/Estabelecimentos_tratado/")

# Extrair datas únicas de início de atividade
df_datas_base = (
    df_estabelecimentos
    .select("DT_INICIO_ATIVIDADE")
    .filter(col("DT_INICIO_ATIVIDADE").isNotNull())
    .distinct()
)

print(f"Total de datas únicas: {df_datas_base.count()}")

In [0]:
# Criar dimensão tempo com cálculos de período
dim_tempo_abertura = (
    df_datas_base
    .withColumn("CO_ANO", year(col("DT_INICIO_ATIVIDADE")))
    .withColumn("CO_MES", month(col("DT_INICIO_ATIVIDADE")))
    .withColumn("CO_TRIMESTRE", ceil(col("CO_MES") / 3))
    .withColumn("CO_SEMESTRE", ceil(col("CO_MES") / 6))
    .withColumn("NO_MES", date_format(col("DT_INICIO_ATIVIDADE"), "MMMM"))
    .withColumn("NO_TRIMESTRE", concat(col("CO_TRIMESTRE"), lit("º Trimestre")))
    .withColumn("NO_SEMESTRE", concat(col("CO_SEMESTRE"), lit("º Semestre")))
    .withColumn("sk_tempo_abertura", sha2(concat_ws("||", col("DT_INICIO_ATIVIDADE")), 256))
    .withColumn("data_criacao", current_timestamp())
    .orderBy("DT_INICIO_ATIVIDADE")
)

print(f"Dimensão TEMPO_ABERTURA criada: {dim_tempo_abertura.count()} registros")

In [0]:
(
    dim_tempo_abertura
    .write
    .format("delta")
    .mode("overwrite")
    .save(f"{path_storage_gold}dim_tempo_abertura/")
)

dim_tempo_abertura.write.format("delta").mode("append").saveAsTable("gold.dim_tempo_abertura")

registrar_transformacao_gold("dim_tempo_abertura", dim_tempo_abertura.count())

print("Dimensão TEMPO_ABERTURA salva em Gold")

In [0]:
%sql 
select * from gold.dim_tempo_abertura limit 10

### Dimensão EMPRESA

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO EMPRESA")
print("=" * 60)

# Carregar tabelas da Silver
df_empresas = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/Empresas_tratado")
df_naturezas = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/natureza_tratado")
df_qualificacoes = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/qualificacoes_tratado")

print(f"EMPRESAS: {df_empresas.count()} registros")
print(f"NATUREZAS: {df_naturezas.count()} registros")
print(f"QUALIFICACOES: {df_qualificacoes.count()} registros")

In [0]:
dim_empresa = df_empresas

In [0]:
# Criar dimensão empresa com joins
dim_empresa = (
    df_empresas.join(
        df_naturezas.select(
            col("CODIGO").alias("CODIGO_NATUREZA"),
            col("DESCRICAO").alias("DESCRICAO_NATUREZA")
        ),
        on=col("NATUREZA_JURIDICA") == col("CODIGO_NATUREZA"),
        how="left"
    )
    .select(
        col("CNPJ_BASICO").alias("CO_CNPJ_BASICO"),
        col("RAZAO_SOCIAL_NOME_EMPRESARIAL").alias("NO_RAZAO_SOCIAL"),
        col("NATUREZA_JURIDICA").alias("NATUREZA_JURIDICA"),
        col("QUALIFICACAO_DO_RESPONSAVEL").alias("QUALIFICACAO_RESPONSAVEL"),
        col("CAPITAL_SOCIAL_DA_EMPRESA").alias("VL_CAPITAL_SOCIAL"),
        col("PORTE_DA_EMPRESA").alias("PORTE_EMPRESA"),
        col("ENTE_FEDERATIVO_RESPONSAVEL").alias("ENTE_FEDERATIVO_RESPONSAVEl")
    )
    .withColumn("sk_empresa", sha2(concat_ws("||", col("CO_CNPJ_BASICO")), 256))
    .withColumn("data_criacao", current_timestamp())
    .dropDuplicates(["CO_CNPJ_BASICO"])
)

print(f"Dimensão EMPRESA criada: {dim_empresa.count()} registros")

In [0]:
dim_empresa.limit(10).display()

In [0]:
(
    dim_empresa
    .write
    .format("delta")
    .mode("overwrite")
    .save(f"{path_storage_gold}dim_empresa/")
)

dim_empresa.write.format("delta").mode("overwrite").saveAsTable("gold.dim_empresa")

registrar_transformacao_gold("dim_empresa", dim_empresa.count())

print("Dimensão EMPRESA salva em Gold")

In [0]:
%sql
select * from gold.dim_empresa limit 10

### Dimensão ESTABELECIMENTO

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO ESTABELECIMENTO")
print("=" * 60)

# Carregar tabelas da Silver
df_estabelecimentos = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/estabelecimentos_tratado")
df_empresas = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/Empresas_tratado")


print(f"ESTABELECIMENTOS: {df_estabelecimentos.count()} registros")
print(f"CNAES: {df_empresas.count()} registros")


In [0]:
df_empresas.columns

In [0]:
df_empresas.columns

In [0]:
# Criar dimensão estabelecimento com joins Join de silver_estabelecimentos + silver_cnaes + silver_municipios.
dim_estabelecimento = (
    df_estabelecimentos
    .join(
        df_empresas.select(
            col("PORTE_DA_EMPRESA").alias("CO_PORTE"),
            col("CAPITAL_SOCIAL_DA_EMPRESA").alias("CAPITAL_SOCIAL_DA_EMPRESA")
        ),
        on=col("CNPJ_BASICO") == col("CNPJ_BASICO"),
        how="left"
    )
    .select(
        col("CNPJ_BASICO").alias("CO_CNPJ_BASICO"),
        col("CNPJ_ORDEM").alias("CO_CNPJ_ORDEM"),
        col("CNPJ_DV").alias("CO_CNPJ_DV"),
        col("UF").alias("SG_UF"),
        col("MUNICIPIO").alias("CO_MUNICIPIO"),
        col("CNAE_FISCAL_PRINCIPAL").alias("CO_CNAE_PRINCIPAL"),
        col("DT_INICIO_ATIVIDADE").alias("DT_INICIO_ATIVIDADE"),
        col("SITUACAO_CADASTRAL").alias("CO_SITUACAO_CADASTRAL")
    )
    .withColumn("sk_estabelecimento", sha2(concat_ws("||", col("CO_CNPJ_BASICO"), col("CO_CNPJ_ORDEM"), col("CO_CNPJ_DV")), 256))
    .withColumn("data_criacao", current_timestamp())
    .dropDuplicates(["CO_CNPJ_BASICO", "CO_CNPJ_ORDEM", "CO_CNPJ_DV"])
)

print(f"Dimensão ESTABELECIMENTO criada: {dim_estabelecimento.count()} registros")

In [0]:
dim_estabelecimento.display()

In [0]:
(
    dim_estabelecimento
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("SG_UF")
    .save(f"{path_storage_gold}dim_estabelecimento/")
)

dim_estabelecimento.write.format("delta").mode("overwrite").saveAsTable("gold.dim_estabelecimento")


registrar_transformacao_gold("dim_estabelecimento", dim_estabelecimento.count())

print("Dimensão ESTABELECIMENTO salva em Gold")

### Dimensão SOCIOS_PERFIL

In [0]:
print("=" * 60)
print("CRIANDO DIMENSÃO SOCIOS_PERFIL")
print("=" * 60)

# Carregar tabelas da Silver
df_socios = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/socios_tratado")
df_qualificacoes = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/qualificacoes_tratado")
df_paises = spark.read.format("delta").load(f"{path_storage_silver}/cnpj/paises_tratado")

print(f"SOCIOS: {df_socios.count()} registros")
print(f"QUALIFICACOES: {df_qualificacoes.count()} registros")
print(f"PAISES: {df_paises.count()} registros")

In [0]:
df_socios.limit(10).display()

In [0]:
df_qualificacoes.limit(10).display()

In [0]:
# Criar dimensão socios com joins qualificacoes 
dim_socios_perfil = (
    df_socios
    .join(
        df_qualificacoes.select(
            col("CODIGO").alias("CODIGO_QUALIFICACAO"),
            col("DESCRICAO").alias("DESCRICAO_QUALIFICACAO")
        ),
        on=col("QUALIFICACAO_DO_SOCIO") == col("CODIGO_QUALIFICACAO"),
        how="left"
    )
    .join(
        df_paises.select(
            col("CODIGO").alias("CODIGO_PAIS"),
            col("DESCRICAO").alias("DESCRICAO_PAIS")
        ),
        on=col("PAIS") == col("CODIGO_PAIS"),
        how="left"
    )
    .select(
        col("CNPJ_BASICO").alias("CO_CNPJ_BASICO"),
        col("IDENTIFICADOR_DO_SOCIO").alias("ID_SOCIO"),
        col("NOME_DO_SOCIO").alias("NO_SOCIO"),
        col("CPF_CNPJ_DO_SOCIO").alias("NU_CPF_CNPJ_SOCIO"),
        col("QUALIFICACAO_DO_SOCIO").alias("CO_QUALIFICACAO"),
        col("DESCRICAO_QUALIFICACAO").alias("NO_QUALIFICACAO"),
        col("FAIXA_ETARIA").alias("NO_FAIXA_ETARIA"),
        col("PAIS").alias("CO_PAIS"),
        col("DESCRICAO_PAIS").alias("NO_PAIS"),
        col("DATA_DE_ENTRADA_NA_SOCIEDADE").alias("DT_ENTRADA_SOCIEDADE")
    )
    .withColumn("sk_socio", sha2(concat_ws("||", col("CO_CNPJ_BASICO"), col("ID_SOCIO")), 256))
    .withColumn("data_criacao", current_timestamp())
    .dropDuplicates(["CO_CNPJ_BASICO", "ID_SOCIO"])
)

print(f"Dimensão SOCIOS_PERFIL criada: {dim_socios_perfil.count()} registros")


In [0]:
dim_socios_perfil.limit(10).display()


In [0]:
(
    dim_socios_perfil
    .write
    .format("delta")
    .mode("overwrite")
    .save(f"{path_storage_gold}dim_socios_perfil/")
)

dim_socios_perfil.write.format("delta").mode("overwite").saveAsTable("gold.dim_socios_perfil")

registrar_transformacao_gold("dim_socios_perfil", dim_socios_perfil.count())

print("Dimensão SOCIOS_PERFIL salva em Gold")

### Fato fato_estabelecimento

In [0]:
print("=" * 60)
print("CRIANDO FATO EMPRESA_DADOS_CADASTRAIS")
print("=" * 60)

# Ler dimensões
df_dim_empresa = spark.read.format("delta").load(f"{path_storage_gold}/dim_empresa")
df_dim_estabelecimento = spark.read.format("delta").load(f"{path_storage_gold}/dim_estabelecimento")





In [0]:
display(dim_estabelecimento)

In [0]:
display(dim_empresa)

In [0]:
# Criar fato com joins às dimensões
fato_estabelecimento = (
    dim_empresa
    .join(
        dim_estabelecimento.select(
            col("CO_CNPJ_BASICO"),
            col("SG_UF").alias("SG_UF_RESPONSAVEL"),
            col("CO_CNAE_PRINCIPAL").alias("CO_CNAE_PRINCIPAL"),
            col("CO_MUNICIPIO").alias("CO_MUNICIPIO"),
            col("CO_SITUACAO_CADASTRAL").alias("CO_SITUACAO_CADASTRAL"),
            col("DT_INICIO_ATIVIDADE"),
            col("sk_estabelecimento").alias("fk_estabelecimento")
        ),
        on="CO_CNPJ_BASICO",
        how="left"
    )
    .select(
        col("CO_CNPJ_BASICO"),
        col("NO_RAZAO_SOCIAL"),
        col("VL_CAPITAL_SOCIAL"),
        col("PORTE_EMPRESA"),
        col("SG_UF_RESPONSAVEL"),
        col("DT_INICIO_ATIVIDADE"),
        col("sk_empresa").alias("fk_empresa"),
        col("fk_estabelecimento"),

        current_timestamp().alias("data_processamento_fato")
    )
    .dropDuplicates(["CO_CNPJ_BASICO",])
)

print(f"Fato EMPRESA_DADOS_CADASTRAIS criado: {fato_estabelecimento.count()} registros")

In [0]:
display(fato_estabelecimento)

In [0]:
(
    fato_estabelecimento
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("SG_UF_RESPONSAVEL")
    .save(f"{path_storage_gold}fato_estabelecimento/")
)

fato_estabelecimento.write.format("delta").mode("overwrite").saveAsTable("gold.fato_estabelecimento")


registrar_transformacao_gold("fato_estabelecimento", fato_estabelecimento.count())

print("Fato fato_estabelecimento salvo em Gold")

### Validações e Estatísticas

In [0]:
print("\nAMOSTRA DIM_EMPRESA:")
spark.table("gold.fato_estabelecimento").limit(30).display()

### Views Analíticas

In [0]:
#spark.sql("""
   """ CREATE OR REPLACE VIEW gold.vw_empresa_completa AS
    SELECT
        e.CO_CNPJ_BASICO,
        e.NO_RAZAO_SOCIAL,
        e.VL_CAPITAL_SOCIAL,
        e.NO_PORTE,
        e.SG_UF_RESPONSAVEL,
        e.NO_NATUREZA_JURIDICA,
        est.NO_FANTASIA,
        est.SG_UF,
        est.NO_MUNICIPIO,
        est.NO_CNAE_PRINCIPAL,
        est.DT_INICIO_ATIVIDADE,
        est.FL_ATIVA,
        est.QT_FILIAIS,
        e.FL_OPCAO_SIMPLES,
        e.FL_OPCAO_MEI,
        e.data_processamento_fato
    FROM gold.fato_empresa_dados_cadastrais e
    LEFT JOIN gold.dim_estabelecimento est
        ON e.CO_CNPJ_BASICO = est.CO_CNPJ_BASICO"""
""")

In [0]:
#spark.sql("""
    """CREATE OR REPLACE VIEW gold.vw_empresa_socios_detalhado AS
    SELECT
        es.CO_CNPJ_BASICO,
        es.NO_RAZAO_SOCIAL,
        es.ID_SOCIO,
        es.NO_SOCIO,
        es.NU_CPF_CNPJ_SOCIO,
        es.NO_QUALIFICACAO,
        es.NO_FAIXA_ETARIA,
        es.NO_PAIS,
        es.DT_ENTRADA_SOCIEDADE,
        e.NO_PORTE,
        e.NO_NATUREZA_JURIDICA,
        e.VL_CAPITAL_SOCIAL,
        e.data_processamento_fato
    FROM gold.fato_empresa_socios es
    LEFT JOIN gold.fato_empresa_dados_cadastrais e
        ON es.CO_CNPJ_BASICO = e.CO_CNPJ_BASICO"""
#""")


In [0]:
#spark.sql("""
    """CREATE OR REPLACE VIEW gold.vw_empresas_por_regiao AS
    SELECT
        est.SG_UF,
        COUNT(DISTINCT e.CO_CNPJ_BASICO) as total_empresas,
        COUNT(DISTINCT est.CO_CNPJ_BASICO) as total_estabelecimentos,
        SUM(e.VL_CAPITAL_SOCIAL) as total_capital_social,
        AVG(e.VL_CAPITAL_SOCIAL) as media_capital_social,
        SUM(CASE WHEN est.FL_ATIVA = true THEN 1 ELSE 0 END) as estabelecimentos_ativos,
        SUM(est.QT_FILIAIS) as total_filiais
    FROM gold.fato_empresa_dados_cadastrais e
    LEFT JOIN gold.dim_estabelecimento est
        ON e.CO_CNPJ_BASICO = est.CO_CNPJ_BASICO
    GROUP BY est.SG_UF"""
#""")


